# Image Classification Neo Compilation Example - Local Mode

This notebook shows an intermediate step in the process of developing an Edge image classification algorithm.

## Notebook Setup

In [1]:
%matplotlib inline

import time
import os
#os.environ["CUDA_VISIBLE_DEVICES"]="0" 
import tensorflow as tf
import numpy as np
import cv2
from tensorflow.python.client import device_lib

tf.__version__


Bad key "text.kerning_factor" on line 4 in
/home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/matplotlib/mpl-data/stylelib/_classic_test_patch.mplstyle.
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.1.3/matplotlibrc.template
or from the matplotlib source distribution


'1.15.3'

## Helper Functions and scripts

In [31]:
def get_img(img_path):
    img = cv2.imread(img_path)
    img = cv2.resize(img.astype(float), (224, 224)) #resize
    imgMean = np.array([104, 117, 124], np.float)
    img -= imgMean #subtract image mean
    return img

In [3]:
def get_label_encode(class_id):
    enc = np.zeros((257),dtype=int)
    enc[class_id] = 1
    return enc

In [4]:
def freeze_graph(model_dir, output_node_names):
    """Extract the sub graph defined by the output nodes and convert 
    all its variables into constant 
    Args:
        model_dir: the root folder containing the checkpoint state file
        output_node_names: a string, containing all the output node's names, 
                            comma separated
    """
    if not tf.gfile.Exists(model_dir):
        raise AssertionError(
            "Export directory doesn't exists. Please specify an export "
            "directory: %s" % model_dir)

    if not output_node_names:
        print("You need to supply the name of a node to --output_node_names.")
        return -1

    # We retrieve our checkpoint fullpath
    checkpoint = tf.train.get_checkpoint_state(model_dir)
    input_checkpoint = checkpoint.model_checkpoint_path
    
    # We precise the file fullname of our freezed graph
    absolute_model_dir = "/".join(input_checkpoint.split('/')[:-1])
    output_graph = absolute_model_dir + "/frozen_model.pb"

    # We clear devices to allow TensorFlow to control on which device it will load operations
    clear_devices = True

    # We start a session using a temporary fresh Graph
    with tf.Session(graph=tf.Graph()) as sess:
        # We import the meta graph in the current default Graph
        saver = tf.train.import_meta_graph(input_checkpoint + '.meta', clear_devices=clear_devices)

        # We restore the weights
        saver.restore(sess, input_checkpoint)

        # We use a built-in TF helper to export variables to constants
        output_graph_def = tf.graph_util.convert_variables_to_constants(
            sess, # The session is used to retrieve the weights
            tf.get_default_graph().as_graph_def(), # The graph_def is used to retrieve the nodes 
            output_node_names.split(",") # The output node names are used to select the usefull nodes
        ) 

        # Finally we serialize and dump the output graph to the filesystem
        with tf.gfile.GFile(output_graph, "wb") as f:
            f.write(output_graph_def.SerializeToString())
        print("%d ops in the final graph." % len(output_graph_def.node))

    return output_graph_def

In [5]:
class VGG(object):
    """alexNet model"""
    def __init__(self, n_classes, batch_size=None):
        self.NUM_CLASSES = n_classes
        self.BATCH_SIZE = batch_size
        self.x = tf.placeholder(tf.float32, [None, 224, 224, 3])
        self.y = tf.placeholder(tf.float32, [None, self.NUM_CLASSES])
        self.buildCNN()

    def buildCNN(self):
        """build model"""
        
        input_layer = self.x
        
        conv1_1 = tf.layers.conv2d(
            input_layer, filters=64, kernel_size=(3, 3), strides=(1, 1), activation=tf.nn.relu, padding='SAME', name='conv1_1'
        )

        conv1_2 = tf.layers.conv2d(
            conv1_1, filters=64, kernel_size=(3, 3), strides=(1, 1), activation=tf.nn.relu, padding='SAME', name='conv1_2'
        )
        pool1 = tf.layers.max_pooling2d(conv1_2, pool_size=(2, 2), strides=(2, 2), padding='SAME', name='pool1')

        # conv2
        conv2_1 = tf.layers.conv2d(
            pool1, filters=128, kernel_size=(3, 3), strides=(1, 1), activation=tf.nn.relu, padding='SAME', name='conv2_1'
        )
        conv2_2 = tf.layers.conv2d(
            conv2_1, filters=128, kernel_size=(3, 3), strides=(1, 1), activation=tf.nn.relu, padding='SAME', name='conv2_2'
        )
        pool2 = tf.layers.max_pooling2d(conv2_2, pool_size=(2, 2), strides=(2, 2), padding='SAME', name='pool2')

        # conv3
        conv3_1 = tf.layers.conv2d(
            pool2, filters=256, kernel_size=(3, 3), strides=(1, 1), activation=tf.nn.relu, padding='SAME', name='conv3_1'
        )
        conv3_2 = tf.layers.conv2d(
            conv3_1, filters=256, kernel_size=(3, 3), strides=(1, 1), activation=tf.nn.relu, padding='SAME', name='conv3_2'
        )
        conv3_3 = tf.layers.conv2d(
            conv3_2, filters=256, kernel_size=(3, 3), strides=(1, 1), activation=tf.nn.relu, padding='SAME', name='conv3_3'
        )
        pool3 = tf.layers.max_pooling2d(conv3_3, pool_size=(2, 2), strides=(2, 2), padding='SAME', name='pool3')

        # conv4
        conv4_1 = tf.layers.conv2d(
            pool3, filters=512, kernel_size=(3, 3), strides=(1, 1), activation=tf.nn.relu, padding='SAME', name='conv4_1'
        )
        conv4_2 = tf.layers.conv2d(
            conv4_1, filters=512, kernel_size=(3, 3), strides=(1, 1), activation=tf.nn.relu, padding='SAME', name='conv4_2'
        )
        conv4_3 = tf.layers.conv2d(
            conv4_2, filters=512, kernel_size=(3, 3), strides=(1, 1), activation=tf.nn.relu, padding='SAME', name='conv4_3'
        )
        pool4 = tf.layers.max_pooling2d(conv4_3, pool_size=(2, 2), strides=(2, 2), padding='SAME', name='pool4')

        # conv5
        conv5_1 = tf.layers.conv2d(
            pool4, filters=512, kernel_size=(3, 3), strides=(1, 1), activation=tf.nn.relu, padding='SAME', name='conv5_1'
        )
        conv5_2 = tf.layers.conv2d(
            conv5_1, filters=512, kernel_size=(3, 3), strides=(1, 1), activation=tf.nn.relu, padding='SAME', name='conv5_2'
        )
        conv5_3 = tf.layers.conv2d(
            conv5_2, filters=512, kernel_size=(3, 3), strides=(1, 1), activation=tf.nn.relu, padding='SAME', name='conv5_3'
        )
        pool5 = tf.layers.max_pooling2d(conv5_3, pool_size=(2, 2), strides=(2, 2), padding='SAME', name='pool5')
        #print('POOL5', pool5.shape)
        #CULPADO flatten = tf.layers.flatten(pool5, name='flatten')
        flatten = tf.reshape(pool5, [-1, 7*7*512])
        fc1_relu = tf.layers.dense(flatten, units=4096, activation=tf.nn.relu, name='fc1_relu')
        fc2_relu = tf.layers.dense(fc1_relu, units=4096, activation=tf.nn.relu, name='fc2_relu')
        self.logits = tf.layers.dense(fc2_relu, units=self.NUM_CLASSES, name='fc3_relu')
        #fc3 = tf.nn.softmax(logits)
        print(self.logits.shape)
        # Return the complete AlexNet model
        return self.logits

In [6]:
net = VGG(257)

Instructions for updating:
Use `tf.keras.layers.Conv2D` instead.
Instructions for updating:
Please use `layer.__call__` method instead.
Instructions for updating:
Use keras.layers.MaxPooling2D instead.
Instructions for updating:
Use keras.layers.Dense instead.
(?, 257)


In [7]:
net.x, net.y, net.logits

(<tf.Tensor 'Placeholder:0' shape=(?, 224, 224, 3) dtype=float32>,
 <tf.Tensor 'Placeholder_1:0' shape=(?, 257) dtype=float32>,
 <tf.Tensor 'fc3_relu/BiasAdd:0' shape=(?, 257) dtype=float32>)

## Dataset

The example is no longer on this URL. Now it has to be downloaded from Google Drive: https://drive.google.com/uc?id=1r6o0pSROcV1_VwT4oSjA2FBUSCWGuxLK

In [ ]:
!conda install -c conda-forge -y gdown
!mkdir -p ../data/Caltech256

In [ ]:
import gdown

url = "https://drive.google.com/uc?id=1r6o0pSROcV1_VwT4oSjA2FBUSCWGuxLK"
destination = "../data/Caltech256/256_ObjectCategories.tar"
gdown.download(url, destination, quiet=False)

In [ ]:
#!curl http://www.vision.caltech.edu/Image_Datasets/Caltech256/256_ObjectCategories.tar -o ../data/Caltech256/256_ObjectCategories.tar

In [ ]:
%%sh
pushd ../data/Caltech256/
tar -xvf 256_ObjectCategories.tar
popd

In [8]:
import glob
import random
from sklearn.model_selection import train_test_split

counter = 0
classes = {}
all_image_paths = list(glob.glob('../data/Caltech256/256_ObjectCategories/*/*.jpg'))
x = []
y = []
for i in all_image_paths:
    _,cat,fname = i.split('/')[3:]
    if classes.get(cat) is None: 
        classes[cat] = counter
        counter += 1

    x.append(i)
    y.append(classes.get(cat))

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.33, random_state=42)

## Training

In [11]:
# (6) Define model's cost and optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=net.logits, labels=net.y))
optimizer = tf.train.AdamOptimizer().minimize(cost)

# (7) Defining evaluation metrics
correct_prediction = tf.equal(tf.argmax(net.logits, 1), tf.argmax(net.y, 1))
accuracy_pct = tf.reduce_mean(tf.cast(correct_prediction, tf.float32)) * 100

# (8) initialize
initializer_op = tf.global_variables_initializer()

epochs = 1
batch_size = 128
test_batch_size = 256

In [14]:
# (9) Run
with tf.Session() as session:
    session.run(initializer_op)
    
    print("Training for", epochs, "epochs.")
    
    # looping over epochs: 
    for epoch in range(epochs):
        # To monitor performance during training
        avg_cost = 0.0 
        avg_acc_pct = 0.0
        
        # loop over all batches of the epoch- 1088 records  
        # batch_size = 128 is already defined
        n_batches = int(len(x_train) / batch_size)
        counter = 1
        for i in range(n_batches):
            # Get the random int for batch
            #random_indices = np.random.randint(len(x_train), size=batch_size) # 1088 is the no of training set records
            pivot = i * batch_size
            feed = {
                net.x: [get_img(i) for i in x_train[pivot:pivot+batch_size]],
                net.y: [get_label_encode(i) for i in y_train[pivot:pivot+batch_size]]
            }
            
            # feed batch data to run optimization and fetching cost and accuracy: 
            _, batch_cost, batch_acc = session.run([optimizer, cost, accuracy_pct], 
                                                   feed_dict=feed)
            # Print batch cost to see the code is working (optional)
            # print('Batch no. {}: batch_cost: {}, batch_acc: {}'.format(counter, batch_cost, batch_acc))
            # Get the average cost and accuracy for all batches: 
            avg_cost += batch_cost / n_batches
            avg_acc_pct += batch_acc / n_batches
            counter += 1
            if counter % 50 == 0:
                print("Batch {}/{}: batch_cost={:.3f}, batch_acc={:.3f},avg_cost={:.3f},avg_acc={:.3f}".format(
                    i,n_batches, batch_cost, batch_acc, avg_cost, avg_acc_pct
                ))
        
        # Get cost and accuracy after one iteration
        test_cost = cost.eval({net.x: [get_img(i) for i in x_test[:test_batch_size]], net.y: [get_label_encode(i) for i in y_test[:test_batch_size]]})
        test_acc_pct = accuracy_pct.eval({net.x: [get_img(i) for i in x_test[:test_batch_size]], net.y: [get_label_encode(i) for i in y_test[:test_batch_size]]})
        # output logs at end of each epoch of training:
        print("Epoch {}: Training Cost = {:.3f}, Training Acc = {:.2f} -- Test Cost = {:.3f}, Test Acc = {:.2f}"\
              .format(epoch + 1, avg_cost, avg_acc_pct, test_cost, test_acc_pct))

    # Getting Final Test Evaluation
    print('\n')
    print("Training Completed. Final Evaluation on Test Data Set.\n") 
    test_cost = cost.eval({net.x: [get_img(i) for i in x_test[:test_batch_size]], net.y: [get_label_encode(i) for i in y_test[:test_batch_size]]})
    test_accy_pct = accuracy_pct.eval({net.x: [get_img(i) for i in x_test[:test_batch_size]], net.y: [get_label_encode(i) for i in y_test[:test_batch_size]]})
    print("Test Cost:", '{:.3f}'.format(test_cost))
    print("Test Accuracy: ", '{:.2f}'.format(test_accy_pct), "%", sep='')
    print('\n')
    
    # Getting accuracy on Validation set  
    val_cost = cost.eval({net.x: [get_img(i) for i in x_test[test_batch_size:test_batch_size*2]], net.y: [get_label_encode(i) for i in y_test[test_batch_size:test_batch_size*2]]})
    val_acc_pct = accuracy_pct.eval({net.x: [get_img(i) for i in x_test[test_batch_size:test_batch_size*2]], net.y: [get_label_encode(i) for i in y_test[test_batch_size:test_batch_size*2]]})
    print("Evaluation on Validation Data Set.\n")
    print("Evaluation Cost:", '{:.3f}'.format(val_cost))
    print("Evaluation Accuracy: ", '{:.2f}'.format(val_acc_pct), "%", sep='')

Training for 1 epochs.
Batch 48/160: batch_cost=5.529, batch_acc=3.906,avg_cost=14.893,avg_acc=0.869
Batch 98/160: batch_cost=5.520, batch_acc=1.562,avg_cost=16.603,avg_acc=1.914
Batch 148/160: batch_cost=5.458, batch_acc=3.125,avg_cost=18.291,avg_acc=3.018
Epoch 1: Training Cost = 18.661, Training Acc = 3.33 -- Test Cost = 5.251, Test Acc = 5.86


Training Completed. Final Evaluation on Test Data Set.

Test Cost: 5.251
Test Accuracy: 5.86%


Evaluation on Validation Data Set.

Evaluation Cost: 5.319
Evaluation Accuracy: 4.30%


## Saving/exporting the model

In [16]:
!rm -rf vgg16/
exporter = tf.saved_model.builder.SavedModelBuilder('vgg16/model')
saver = tf.train.Saver()
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    img = get_img('TestImage.jpg')
    batch = np.array([img for i in range(batch_size)]).reshape((batch_size,224,224,3))
    print(batch.shape)
    x = net.x
    y = net.logits
    start_time = time.time()
    out = sess.run(y, feed_dict = {x: batch})
    print("Elapsed time: {}".format(time.time() - start_time))
    
    exporter.add_meta_graph_and_variables(
        sess, 
        tags=[tf.saved_model.tag_constants.SERVING], 
        signature_def_map={
            tf.saved_model.signature_constants.DEFAULT_SERVING_SIGNATURE_DEF_KEY:
            tf.saved_model.signature_def_utils.predict_signature_def(
                inputs={"inputs": net.x}, 
                outputs={"outputs": net.logits }
            )
        },
        strip_default_attrs=True)
    #exporter.save()
    saver.save(sess, 'vgg16/model/model')

(128, 224, 224, 3)
Elapsed time: 0.35367631912231445
Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.utils.build_tensor_info or tf.compat.v1.saved_model.build_tensor_info.
INFO:tensorflow:No assets to save.
INFO:tensorflow:No assets to write.


In [17]:
_ = freeze_graph('vgg16/model', 'fc3_relu/BiasAdd')

INFO:tensorflow:Restoring parameters from vgg16/model/model
Instructions for updating:
Use `tf.compat.v1.graph_util.convert_variables_to_constants`
Instructions for updating:
Use `tf.compat.v1.graph_util.extract_sub_graph`
INFO:tensorflow:Froze 32 variables.
INFO:tensorflow:Converted 32 variables to const ops.
119 ops in the final graph.


In [18]:
!cd vgg16 && rm -f model.tar.gz && cd model && tar -czvf ../model.tar.gz frozen_model.pb

frozen_model.pb


In [19]:
import time
import sagemaker
import os
import json
import boto3

# Retrieve the default bucket
sagemaker_session = sagemaker.Session()
default_bucket = sagemaker_session.default_bucket()

In [22]:
type(sagemaker_session)

sagemaker.session.Session

In [23]:
role=sagemaker.session.get_execution_role()
job_prefix='VGG16'
path='neo/%s' % job_prefix

sm = boto3.client('sagemaker')
!aws s3 cp vgg16/model.tar.gz s3://$default_bucket/$path/model.tar.gz

upload: vgg16/model.tar.gz to s3://sagemaker-us-east-1-160951647621/neo/VGG16/model.tar.gz


In [25]:
target_device='ml_m5'
# 'lambda'|'ml_m4'|'ml_m5'|'ml_c4'|'ml_c5'|'ml_p2'|'ml_p3'|'ml_g4dn'|'ml_inf1'|'jetson_tx1'|'jetson_tx2'|'jetson_nano'|'jetson_xavier'|
# 'rasp3b'|'imx8qm'|'deeplens'|'rk3399'|'rk3288'|'aisage'|'sbe_c'|'qcs605'|'qcs603'|'sitara_am57x'|'amba_cv22'|'x86_win32'|'x86_win64'

In [26]:
job_name="%s-%d" % (job_prefix, int(time.time()))
sm.create_compilation_job(
    CompilationJobName=job_name,
    RoleArn=role,
    InputConfig={
        'S3Uri': "s3://%s/%s/model.tar.gz" % (default_bucket, path),
        'DataInputConfig': '{"data":[1,224,224,3]}',
        'Framework': 'TENSORFLOW'
    },
    OutputConfig={
        'S3OutputLocation': "s3://%s/%s/" % (default_bucket, path),
        'TargetDevice': target_device
    },
    StoppingCondition={
        'MaxRuntimeInSeconds': 300
    }
)
print(job_name)

VGG16-1601381999


In [29]:
default_bucket

'sagemaker-us-east-1-160951647621'

## NEO

In [30]:
!echo s3://$default_bucket/neo/VGG16/model-$target_device\.tar.gz
!aws s3 cp s3://$default_bucket/neo/VGG16/model-$target_device\.tar.gz .

s3://sagemaker-us-east-1-160951647621/neo/VGG16/model-ml_m5.tar.gz
fatal error: An error occurred (404) when calling the HeadObject operation: Key "neo/VGG16/model-ml_m5.tar.gz" does not exist


In [ ]:
!tar -tzvf model-$target_device\.tar.gz

In [ ]:
!rm -rf neo_test && mkdir neo_test
!tar -xzvf model-$target_device\.tar.gz -C neo_test

In [ ]:
!ldd neo_test/compiled.so

In [ ]:
%%time
import os
import numpy as np
import cv2
import time
from dlr import DLRModel

model_path='neo_test'
imgMean = np.array([104, 117, 124], np.float)
img = cv2.imread("TestImage.jpg")
img = cv2.resize(img.astype(float), (224, 224)) # resize
img -= imgMean #subtract image mean
img = img.reshape((1, 224, 224, 3))

device = 'cpu'                           # Go, Raspberry Pi, go!
model = DLRModel(model_path, dev_type=device)

print(model.get_input_names())

def predict(img):
    start = time.time()
    input_data = {'Placeholder': img}
    out = model.run(input_data)
    return (out, time.time()-start)
startTime = time.time()
out = [predict(img)[1] for i in (range(1))]
print("Elapsed time: {}".format((time.time() - start_time)))
#top1 = np.argmax(out[0])
#prob = np.max(out)
#print("Class: %d, probability: %f" % (top1, prob))

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

fig = plt.figure(figsize=(20,10))
ax = plt.axes()

ax.plot(out)